# MS Teams archive tool - about

This is MS Teams archive tool that saves the conversation of chats and teams.
It is written for archiving the MS Teams chats of Lodz University of Technology, but probably may be easily adapted for other cases.

## Requirements

Following packages are required:
* Python 3
* Selenium
* gecodriver (firefox)

Gecodriver (binary file) should be stored in `/bin` location (for conda it is `/home/[USER]/.conda/envs/[ENV_NAME]/bin`).

When completed there should be a number of files created in `output_path`, including screenshots and texts. The texts require post-processing to look nicely. Nevertheless, may be considered as archive.

The output of the script should look like below (Note that for Teams it only supports the "General" channel):

```
[output_path]
├── chats
│   ├── Adam aaa, Adam aaa, Alicja aaa, Bartosz aaa, ...
│   │   ├── screenshots
│   │   │   ├── 0.png
│   │   │   ├── 1.png
│   │   │   ├── 2.png
│   │   │   ├── ...
│   │   └── txt
│   │       ├── 0.txt
│   │       ├── 1.txt
│   │       ├── 2.txt
│   │       ├── ...
│   ├── ...
│   │   ├── screenshots
│   │   │   ├── 0.png
│   │   │   ├── 1.png
│   │   │   ├── 2.png
│   │   │   └── ...
│   │   └── txt
│   │       ├── 0.txt
│   │       ├── 1.txt
│   │       ├── 2.txt
│   │       └── ...
├── teams
│   ├── 20-2020-ETE2_General
│   │   ├── screenshots
│   │   │   ├── 0.png
│   │   │   ├── 1.png
│   │   │   ├── 2.png
│   │   │   ├── ...
│   │   └── txt
│   │       ├── 0.txt
│   │       ├── 1.txt
│   │       ├── 2.txt
│   │       ├── ...
│   ├── 20-2020-SMPD-01_General
│   │   ├── screenshots
│   │   │   └── 0.png
│   │   └── txt
│   │       └── 0.txt
│   ├── ...

```

*Note: This script is not supported and may not work in future.*


# Configuration

In [ ]:
import os, time, selenium
from selenium import webdriver
from tqdm import tqdm
import re

In [ ]:
url = "https://teams.microsoft.com/_#/school/conversations/General"
output_path = '/tmp'
email = 'PROVIDE YOUR EMAIL ADDRESS'
password = 'PROVIDE YOUR PASSWORD'

In [ ]:
options = webdriver.FirefoxOptions()
#options.add_argument("--headless")
driver = webdriver.Firefox(options=options)
driver.set_window_size(1440, 1200)

In [ ]:
driver.get(url) # this may take some time
time.sleep(30) 

# Prepare MS Teams - provide credentials, etc.

## Provide username (email)

In [ ]:
if 'authorize' in driver.current_url:
    login_fmt = driver.find_element_by_name("loginfmt")
    login_fmt.send_keys(email)
    login_fmt.send_keys(u'\ue007') # hit enter
    time.sleep(10)

## CAS Form

In [ ]:
if 'cas' in driver.current_url:
    username_field = driver.find_element_by_name("username")
    password_field = driver.find_element_by_name("password")
    username_field.send_keys(email)
    password_field.send_keys(password)
    password_field.send_keys(u'\ue007')
    time.sleep(10)

## Back to Teams, click stay signed in

In [ ]:
try:
    driver.find_element_by_xpath('//input[@type="submit"]').click()
    time.sleep(10)
except Exception as ex:
    print(f'WARNING: {ex}')

## Answer to use web app instead of dektop / phone app

In [ ]:
try:
    driver.find_element_by_xpath('//a[@class="use-app-lnk"]').click()
    time.sleep(10)
except Exception as ex:
    print(f'WARNING: {ex}')

## Dismiss pop-up notification

In [ ]:
try:
    driver.find_element_by_xpath('//button[@title="Dismiss"]').click()
    time.sleep(1)
except Exception as ex:
    print(f'WARNING: {ex}')

# Go through all chats, take screenshots and save text

In [ ]:
driver.find_element_by_xpath('//button[@aria-label="Chat Toolbar"]').click()
time.sleep(2)

In [ ]:
print('Saving chats...')

chat_divs = driver.find_elements_by_class_name('recipient-group-list-item')
for i, chat_div in enumerate(tqdm(chat_divs)):
    time.sleep(2); chat_div.click()
    
    foldername = chat_div.get_attribute('data-tid').replace('chat-li-entry-with-', '')[:100]
    os.makedirs(f'{output_path}/chats/{foldername}', exist_ok=True)
    os.makedirs(f'{output_path}/chats/{foldername}/screenshots', exist_ok=True)
    os.makedirs(f'{output_path}/chats/{foldername}/txt', exist_ok=True)
    
    j = 0
    while(True):
        before_scroll = driver.get_screenshot_as_base64()
        driver.save_screenshot(f'{output_path}/chats/{foldername}/screenshots/{j}.png')
        
        chat_converation = driver.find_element_by_tag_name('messages-header').text
        with open(f'/{output_path}/chats/{foldername}/txt/{j}.txt', 'wt') as f:
            f.write(chat_converation)
        
        # next page - scroll
        time.sleep(2)
        try:
            driver.find_element_by_xpath('//virtual-repeat[@vr-item-name="thread"]').send_keys(selenium.webdriver.common.keys.Keys.PAGE_UP)
        except Exception:
            break
        after_scroll = driver.get_screenshot_as_base64()
        if before_scroll == after_scroll: # break if nothing changed after scroll
            break
        j += 1

        

# The same for teams

In [ ]:
print('Saving Teams conversation...')
driver.find_element_by_xpath('//button[@aria-label="Teams Toolbar"]').click()

# only "General" chats
chat_divs = driver.find_elements_by_xpath('//li[@acc-role-dom="tree-node" and @role="treeitem" and @aria-level="2"]')
for i, chat_div in enumerate(tqdm(chat_divs)):
    time.sleep(2); chat_div.click()
    
    foldername = re.sub(r'\s+', '_', chat_div.text)[:100]
    os.makedirs(f'{output_path}/teams/{foldername}', exist_ok=True)
    os.makedirs(f'{output_path}/teams/{foldername}/screenshots', exist_ok=True)
    os.makedirs(f'{output_path}/teams/{foldername}/txt', exist_ok=True)
    
    j = 0
    while(True):
        before_scroll = driver.get_screenshot_as_base64()
        driver.save_screenshot(f'{output_path}/teams/{foldername}/screenshots/{j}.png')
        
        chat_converation = driver.find_element_by_tag_name('messages-header').text
        with open(f'/{output_path}/teams/{foldername}/txt/{j}.txt', 'wt') as f:
            f.write(chat_converation)
        
        # next page - scroll
        time.sleep(2)
        try:
            driver.find_element_by_xpath('//virtual-repeat[@vr-item-name="thread"]').send_keys(selenium.webdriver.common.keys.Keys.PAGE_UP)
        except Exception:
            break
        after_scroll = driver.get_screenshot_as_base64()
        if before_scroll == after_scroll: # break if nothing changed after scroll
            break
        j += 1

# Finish

In [ ]:
driver.quit()